单任务测试集

In [14]:
from class_uhpc import *
import numpy as np
import xgboost as xgb
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [15]:
task_names = [
    'Flowability',
    'Porosity',
    'Compressive strength',
    'Flexural strength'
]

In [18]:
for task_name in task_names:
    ds = DataSet_uhpc(path='./', num_feature=None, test_size=0.2, seed=42, task_name=task_name)
    X_train, y_train = ds.get_train()
    X_test, y_test = ds.get_test()
    print('='*50)
    print(ds.get_name())
    model = xgb.XGBRegressor()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    r2 = r2_score(y_test, y_pred)
    print(r2)
    mae = mean_absolute_error(y_test, y_pred)
    print(mae)
    mse = mean_squared_error(y_test, y_pred)
    print(mse)


Flowability


XGBoostError: [11:36:09] C:\actions-runner\_work\xgboost\xgboost\src\data\data.cc:565: Check failed: valid: Label contains NaN, infinity or a value too large.